# PlanPilot AI — ML Model Training

This notebook trains the XGBoost planning application approval prediction model.

**Prerequisites (run locally before this notebook):**
```bash
brew install gdal
pip install -r requirements.txt

python scripts/setup_db.py
python scripts/ingest_flood.py --file data/flood/RoFRS_London.shp
python scripts/ingest_conservation.py --file data/conservation/Conservation_Areas.shp
python scripts/ingest_greenbelt.py --file data/greenbelt/England_Green_Belt_2024_25_WGS84.shp
python scripts/ingest_article4.py --file data/article4/article4_directions.geojson
python scripts/ingest_price_paid.py --csv data/price_paid/pp-complete.csv --postcodes data/postcodes/ONSPD_NOV_2025_UK.csv
python scripts/ingest_ibex.py --las E09000028 --postcodes data/postcodes/ONSPD_NOV_2025_UK.csv --years 5
```

**Notebook steps:**
1. Install dependencies
2. Connect to Supabase
3. Check data readiness
4. Run feature engineering
5. Train XGBoost
6. Evaluate (ROC-AUC + feature importance)
7. Download `planning_model.pkl`

## 1. Install dependencies

In [ ]:
!pip install asyncpg xgboost scikit-learn joblib pandas numpy matplotlib seaborn python-dotenv -q

## 2. Connect to Supabase

Enter your Supabase **direct Postgres connection string** from:
> Supabase Dashboard → Project → Settings → Database → Connection string → URI

Make sure to use the **direct connection** (port 5432), not the pooler.

In [ ]:
import asyncpg
import asyncio
import pandas as pd
import numpy as np

# Paste your Supabase direct Postgres URI here
DATABASE_URL = "postgresql://postgres:<password>@db.<ref>.supabase.co:5432/postgres"

async def test_connection():
    conn = await asyncpg.connect(DATABASE_URL)
    version = await conn.fetchval("SELECT version()")
    postgis = await conn.fetchval("SELECT PostGIS_Version()")
    await conn.close()
    print(f"Connected: {version[:50]}")
    print(f"PostGIS version: {postgis}")

await test_connection()

## 3. Check data readiness

Verify that all ingestion scripts have been run successfully.

In [ ]:
async def check_tables():
    conn = await asyncpg.connect(DATABASE_URL)

    checks = [
        ("flood_zones",            "SELECT COUNT(*) FROM flood_zones"),
        ("conservation_areas",     "SELECT COUNT(*) FROM conservation_areas"),
        ("greenbelt_areas",        "SELECT COUNT(*) FROM greenbelt_areas"),
        ("article4_zones",         "SELECT COUNT(*) FROM article4_zones"),
        ("price_paid",             "SELECT COUNT(*) FROM price_paid"),
        ("planning_applications",  "SELECT COUNT(*) FROM planning_applications"),
        ("features computed",      "SELECT COUNT(*) FROM planning_applications WHERE flood_zone IS NOT NULL"),
        ("approved",               "SELECT COUNT(*) FROM planning_applications WHERE decision = 'approved'"),
        ("refused",                "SELECT COUNT(*) FROM planning_applications WHERE decision = 'refused'"),
    ]

    print(f"{'Table':<30} {'Count':>10}")
    print("-" * 42)
    for label, query in checks:
        try:
            count = await conn.fetchval(query)
            status = "✅" if count and count > 0 else "❌ EMPTY"
            print(f"{label:<30} {count:>10,}  {status}")
        except Exception as e:
            print(f"{label:<30} {'ERROR':>10}  ❌ {e}")

    await conn.close()

await check_tables()

## 4. Feature engineering

For each planning application row that doesn't yet have features computed,
this queries the spatial layers and market data to populate the 10 feature columns.

**Skip this cell if `features computed` count above already equals `planning_applications` count.**

In [ ]:
CONSTRAINT_QUERY = """
    SELECT
        COALESCE(
            (SELECT zone_number FROM flood_zones
             WHERE ST_Contains(geom, app.geom)
             ORDER BY zone_number DESC LIMIT 1),
            1
        ) AS flood_zone,
        EXISTS(
            SELECT 1 FROM conservation_areas WHERE ST_Contains(geom, app.geom)
        ) AS in_conservation_area,
        EXISTS(
            SELECT 1 FROM greenbelt_areas WHERE ST_Contains(geom, app.geom)
        ) AS in_greenbelt,
        EXISTS(
            SELECT 1 FROM article4_zones WHERE ST_Contains(geom, app.geom)
        ) AS in_article4_zone
    FROM (SELECT ST_SetSRID(ST_MakePoint($1, $2), 4326) AS geom) app
"""

PLANNING_METRICS_QUERY = """
    SELECT
        COUNT(*) FILTER (WHERE decision = 'approved')::float /
            NULLIF(COUNT(*), 0) AS local_approval_rate,
        AVG(decision_days) AS avg_decision_time_days,
        COUNT(*) AS similar_applications_nearby
    FROM planning_applications
    WHERE ST_DWithin(
        geom::geography,
        ST_SetSRID(ST_MakePoint($1, $2), 4326)::geography,
        500
    )
    AND decision_date < $3
    AND decision_date >= $3 - INTERVAL '5 years'
"""

MARKET_QUERY = """
    SELECT
        AVG(price_per_m2) AS avg_price_per_m2,
        (
            AVG(price_per_m2) FILTER (WHERE sale_date >= $3::date - INTERVAL '12 months') -
            AVG(price_per_m2) FILTER (WHERE sale_date BETWEEN $3::date - INTERVAL '24 months'
                                                           AND $3::date - INTERVAL '12 months')
        ) /
        NULLIF(
            AVG(price_per_m2) FILTER (WHERE sale_date BETWEEN $3::date - INTERVAL '24 months'
                                                           AND $3::date - INTERVAL '12 months'),
            0
        ) AS price_trend_24m
    FROM price_paid
    WHERE ST_DWithin(
        geom::geography,
        ST_SetSRID(ST_MakePoint($1, $2), 4326)::geography,
        500
    )
    AND sale_date BETWEEN $3::date - INTERVAL '24 months' AND $3::date
"""

UPDATE_QUERY = """
    UPDATE planning_applications SET
        flood_zone                  = $1,
        in_conservation_area        = $2,
        in_greenbelt                = $3,
        in_article4_zone            = $4,
        local_approval_rate         = $5,
        avg_decision_time_days      = $6,
        similar_applications_nearby = $7,
        avg_price_per_m2            = $8,
        price_trend_24m             = $9,
        avg_epc_rating              = $10
    WHERE id = $11
"""

async def run_feature_engineering(batch_size=200):
    conn = await asyncpg.connect(DATABASE_URL)

    apps = await conn.fetch("""
        SELECT id, ST_X(geom) AS lon, ST_Y(geom) AS lat, decision_date
        FROM planning_applications
        WHERE geom IS NOT NULL
          AND decision_date IS NOT NULL
          AND flood_zone IS NULL
        ORDER BY id
    """)

    total = len(apps)
    print(f"Computing features for {total:,} applications...")
    if total == 0:
        print("Nothing to do — all rows already have features.")
        await conn.close()
        return

    done, errors = 0, 0
    batch = []

    for app in apps:
        try:
            c = await conn.fetchrow(CONSTRAINT_QUERY, app['lon'], app['lat'])
            p = await conn.fetchrow(PLANNING_METRICS_QUERY, app['lon'], app['lat'], app['decision_date'])
            m = await conn.fetchrow(MARKET_QUERY, app['lon'], app['lat'], app['decision_date'])

            batch.append((
                int(c['flood_zone']),
                bool(c['in_conservation_area']),
                bool(c['in_greenbelt']),
                bool(c['in_article4_zone']),
                round(float(p['local_approval_rate'] or 0.0), 4),
                round(float(p['avg_decision_time_days'] or 0.0), 1),
                int(p['similar_applications_nearby'] or 0),
                round(float(m['avg_price_per_m2'] or 0.0), 2),
                round(float(m['price_trend_24m'] or 0.0), 4),
                'D',  # EPC default — populated by EPC API at inference time
                app['id'],
            ))
            done += 1
        except Exception as e:
            errors += 1

        if len(batch) >= batch_size:
            await conn.executemany(UPDATE_QUERY, batch)
            batch.clear()
            print(f"  {done:,}/{total:,} done  |  {errors} errors", end='\r')

    if batch:
        await conn.executemany(UPDATE_QUERY, batch)

    await conn.close()
    print(f"\nFeature engineering complete. Updated: {done:,}  |  Skipped: {errors}")

await run_feature_engineering()

## 5. Load training data

In [ ]:
EPC_MAP = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}

FEATURE_COLS = [
    'flood_zone',
    'in_conservation_area',
    'in_greenbelt',
    'in_article4_zone',
    'local_approval_rate',
    'avg_decision_time_days',
    'similar_applications_nearby',
    'avg_price_per_m2',
    'price_trend_24m',
    'epc_score',
]

async def load_training_data():
    conn = await asyncpg.connect(DATABASE_URL)
    rows = await conn.fetch("""
        SELECT
            flood_zone,
            in_conservation_area::int,
            in_greenbelt::int,
            in_article4_zone::int,
            local_approval_rate,
            avg_decision_time_days,
            similar_applications_nearby,
            avg_price_per_m2,
            price_trend_24m,
            avg_epc_rating,
            CASE WHEN decision = 'approved' THEN 1 ELSE 0 END AS approved
        FROM planning_applications
        WHERE flood_zone IS NOT NULL
          AND local_approval_rate IS NOT NULL
    """)
    await conn.close()
    return pd.DataFrame([dict(r) for r in rows])

df = await load_training_data()
df['epc_score'] = df['avg_epc_rating'].map(EPC_MAP).fillna(4)

print(f"Total records: {len(df):,}")
print(f"Class balance: {df['approved'].mean():.1%} approved / {1-df['approved'].mean():.1%} refused")
print(f"\nFeature summary:")
df[FEATURE_COLS].describe().round(3)

## 6. Train XGBoost model

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import joblib

X = df[FEATURE_COLS].values
y = df['approved'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {len(X_train):,} rows")
print(f"Test set:     {len(X_test):,} rows")

model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss',
    random_state=42,
    verbosity=1,
)

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=50,
)

print("\nTraining complete.")

## 7. Evaluate

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import RocCurveDisplay

y_proba = model.predict_proba(X_test)[:, 1]
y_pred  = model.predict(X_test)
auc     = roc_auc_score(y_test, y_proba)

print(f"ROC-AUC: {auc:.4f}")
print()
print(classification_report(y_test, y_pred, target_names=['Refused', 'Approved']))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# --- Feature importance ---
importance = pd.Series(model.feature_importances_, index=FEATURE_COLS).sort_values()
importance.plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Feature Importances')
axes[0].set_xlabel('Importance score')

# --- ROC curve ---
RocCurveDisplay.from_predictions(y_test, y_proba, ax=axes[1], name=f'XGBoost (AUC={auc:.3f})')
axes[1].set_title('ROC Curve')
axes[1].plot([0,1],[0,1],'k--')

# --- Confusion matrix ---
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', ax=axes[2],
            xticklabels=['Refused','Approved'],
            yticklabels=['Refused','Approved'],
            cmap='Blues')
axes[2].set_title('Confusion Matrix')
axes[2].set_ylabel('Actual')
axes[2].set_xlabel('Predicted')

plt.tight_layout()
plt.savefig('model_evaluation.png', dpi=150)
plt.show()
print("Saved: model_evaluation.png")

## 8. Save and download the model

After downloading, place `planning_model.pkl` at `backend/ml/planning_model.pkl`.

In [ ]:
import joblib
from google.colab import files

MODEL_PATH = 'planning_model.pkl'
joblib.dump(model, MODEL_PATH)
print(f"Model saved: {MODEL_PATH}")

# Download to your local machine
files.download(MODEL_PATH)
files.download('model_evaluation.png')

print(f"\nNext step: move planning_model.pkl to backend/ml/planning_model.pkl")

## 9. (Optional) Verify the model matches the backend feature order

The backend `ml.py` service must use features in the **exact same order** as trained here.
Run this to confirm.

In [ ]:
BACKEND_FEATURE_ORDER = [
    'flood_zone',
    'in_conservation_area',
    'in_greenbelt',
    'in_article4_zone',
    'local_approval_rate',
    'avg_decision_time_days',
    'similar_applications_nearby',
    'avg_price_per_m2',
    'price_trend_24m',
    'epc_score',
]

assert FEATURE_COLS == BACKEND_FEATURE_ORDER, (
    f"Feature order mismatch!\nNotebook: {FEATURE_COLS}\nBackend:  {BACKEND_FEATURE_ORDER}"
)

# Quick sanity-check prediction
test_input = np.array([[1, 0, 0, 0, 0.72, 65.0, 8, 9500.0, 0.03, 5]])
prob = model.predict_proba(test_input)[0][1]
print(f"✅ Feature order verified")
print(f"Sample prediction (no constraints, 72% local approval rate): {prob:.1%} approval probability")